## -=Подготовка к запуску=-

### Установка пакетов

В случае, если пакеты через requirements.тхт установлены не были, блок ниже установит все необходимые пакеты для работы данного блокнота.

In [ ]:
%pip install ultralytics==8.0.198
%pip install Pillow==10.0.1
%pip install numpy==1.24.3
%pip install opencv-python==4.6.0.66

### Импорт библиотек

In [ ]:
import cv2
import numpy as np
from PIL import Image
from ultralytics import YOLO
import os

Импорт функций из detectors.py, в данном файле находятся все функции связанные с детекцией и моделями, они вынесены в отдельный файл для удобства работы.

In [ ]:
import detectors

Импорт вспомогательных функций из utils.py, вынесены вотдельный файл так же для удобства.

In [ ]:
import utils

## -=Загрузка моделей=-

In [ ]:
model0 = YOLO("models/seg_n_aug.pt")
model1 = YOLO("models/cls_seg_garb_aug.pt")
model2 = YOLO("models/cls_truck.pt")
model3 = YOLO("models/det_garbage_aug.pt")
model4 = YOLO("models/cls_correct.pt")
model5 = YOLO("models/cls_det_garbage_n_aug.pt")
model_ans_det1 = YOLO("models/cls_ans_det1.pt") #n
model_ans_det2 = YOLO("models/cls_ans_det2.pt") #n
model_ans_det3 = YOLO("models/cls_ans_det3.pt") #m
model_ans_seg1 = YOLO("models/cls_ans_seg1.pt") #n
model_ans_seg2 = YOLO("models/cls_ans_seg2.pt") #n
model_ans_seg3 = YOLO("models/cls_ans_seg3.pt") #m
models_all = [model0, model1, model2, model3, model4, model5, model_ans_det1, model_ans_det2, model_ans_det3, model_ans_seg1, model_ans_seg2, model_ans_seg3]

## -=Общая информация=-
### Все функции связанные с запуском моделей находятся в detectors.py

### Мы разработали два "пути" моеделей, каждый включает в себя обработку пятью моделями
Оба пути включают в себя первые две модели-классификатора, 
первая оценивает валидность фотографии для оценивания, 
вторая - валидность расположения грузовика для оценки.
Если изображение не проходит хотя бы одну из этих моделей - дальнейшая детекция не происходит, для экономии времени.
#### Первый путь - "быстрый", включает в себя детекцию, а затем классификацию
#### Второй путь - "точный", включает в себя сегментацию, а затем классификацию
Также у обоих путей можно включить дополнительную оценку ансамблем, это увеличивает длительность оценки, но повышает точность за счет задействования дополнительных классификаторов.

## -=Обработка видео=-

### Обработка видео равноценно обработке множества кадров этого видео

Разобъём видео на кадры в рамках указанного интервала, для этого воспользуемся функцией из utils

In [ ]:
#Функция возвращает адрес внременного хранилища фреймов, так как если выставить большой промежуток, фреймов становится много и они забивают ОЗУ
#А также функция возвращает ФПС, вдруг пригодится
frames_folder, fps = utils.video_to_frames("examples/3336399.mp4", 110, 130, 20)

Получим пути к кадрам для работы с ними

In [ ]:
frames_paths = []

for root, dirs, files in os.walk(frames_folder):
    for file in files:
        frames_paths.append(os.path.join(root, file))

print(frames_paths)

Проверим, подходит ли содержимое фреймов для оценки, или видео неккоректно
#### 0 - кадр корректен, 1 - нет

In [ ]:
results_list = []
for frame in frames_paths:
    img, results = detectors.run_model(models_all, frame, "class_correct")
    results_list.append(results[0].probs.top1)
results_list

Как мы видим, все фреймы прошли первую проверку, но на всякий случай отсортируем их

In [ ]:
first_check_list = []
for idx, result in enumerate(results_list):
    if result == 0:
        first_check_list.append(frames_paths[idx])
first_check_list

Проверим, на всех ли кадрах хорошо виден кузов
#### 1 - да, 0 - нет

In [ ]:
results_list = []
for frame in first_check_list:
    img, results = detectors.run_model(models_all, frame, "class_truck")
    results_list.append(results[0].probs.top1)
results_list

Как можно заметить, кадров на которых система может разглядеть содержимое не так уж и много, даже проявилась аномалия, отберём фреймы

In [ ]:
second_check_list = []
for idx, result in enumerate(results_list):
    if result == 1:
        second_check_list.append(frames_paths[idx])
second_check_list

### Есть два способа использования моделей - каждой по отдельности, либо же использовать функцию с уже скомбинированным применением, сначала пройдёмся по одиночным использованиям.

## -=Сегментация-Классификация-Ансамбль=-

### Сегментация содержимого кузова и его классификация
### Для того чтобы провести максимально точный анализ, необходимо проводить анализ всех фереймов и выбирать итоговый результат по болшинству.

Загрузка изображений

In [ ]:
images = []
for frame in second_check_list:
    image = Image.open(frame)
    images.append(image)
#Покажем какое-нибудь загруженное изображение, его в дальнейшем и будем показывать
display(images[-1])

Сегментация

In [ ]:
segmentations_list = []
for image in images:
    image, results = detectors.run_model(models_all, image, "segment")
    segmentations_list.append(results)
display(image)


Получение маски берёт результаты полученные в сегментации

In [ ]:
masks_list = []
for result in segmentations_list:
    mask = detectors.get_mask(result)
    mask = Image.fromarray(mask)
    masks_list.append(mask)
display(mask)

Получение прогноза классификатора из масок

In [ ]:
classifications = []
for mask in masks_list:
    cls_seg_image, cls_seg_results = detectors.run_model(models_all, mask, "class_segment")
    classifications.append(cls_seg_results[0].probs.top1)
print(classifications)
cls_seg_image

Так как кадров несколько, увеличим точность за счет голосования, в данном случае это не имеет особого смысла, но в случае, если бы "мнения" расходились у меньшинства, большинство бы свела ответ в верную сторону

In [ ]:
most_common_class = utils.most_common(classifications)
most_common_class

Как и ожидалось, классификаторы оценили всё как 1, т.е. бетон, что правильно для данного видео

### Приминение ансамбля

Применение модуля ансамблирования может повысить точность, пусть сейчас это и не имеет особого смысла, я покажу как его можно применить

В данном проекте имеются два ансамбля, один для детектора, второй для сегментатора, это связано с тем, что их классификаторы обучались на отличающихся данных и использование единого ансамбля может навредить точности, если дополнительно не конфигрировать "вес" выводов.

In [ ]:
#Используются маски для классификации
results_list = []
for mask in masks_list:
    result = detectors.ensemble_detect(models_all, mask, "segment")
    results_list.append(result)
utils.most_common(results_list)

## -=Детекция-Классификация-Ансамбль=-

### Детекция содержимого кузова и его классификация
В случае детекции всё рактически то же самое, только вместо сегментирования используется детекция, что позволяет повысить точность не используя ансамбль, но детекция сильнее подвержена ложным срабатываниям

Все этапы те же самые, потому коментариев в этой части меньше

In [ ]:
images = []
for frame in second_check_list:
    image = Image.open(frame)
    images.append(image)
#Покажем какое-нибудь загруженное изображение, его в дальнейшем и будем показывать
display(images[-1])

Детекция

In [ ]:
detections_list = []
detection_results = []
for image in images:
    image, results = detectors.run_model(models_all, image, "detect")
    detections_list.append(results)
    detection_results.append(results[0].boxes.cls[0].cpu().numpy().astype(int))
main_detection_result = utils.most_common(detection_results)
#main_result понадобится для дальнейшего голосования
print(main_detection_result)
display(image)


Как видно из изображения, детектор уже готов сказать, чем является содержимое кузова и все детекторы говорят одно и то же

Получение кропа берёт переменные полученные в детекции

In [ ]:
crops_list = []
for result in detections_list:
    crop = detectors.get_crop(result)
    crop = Image.fromarray(crop)
    crops_list.append(crop)
display(crop)

Получение прогноза классификатора

In [ ]:
classifications = []
for crop in crops_list:
    cls_det_image, cls_det_results = detectors.run_model(models_all, crop, "class_detect")
    classifications.append(cls_det_results[0].probs.top1)
print(classifications)
cls_det_image

Система может вынести обобщённый ответ и без ассамблеи, но детектор обладает тенденцией делать оишбочные показания, потому этот метод более быстрый, но менее точный.

Пусть в этот раз нам и повезло, но это связано с тем, что изображения достаточно качественные.

In [ ]:
classifications.append(main_detection_result)
result = utils.most_common(classifications)
result

Ансамблирование делать нет смысла, так как его запуск будет отличаться только одним параметром, вместо "segment" нужно будет написать "detect"

## -=Объединённые функции=-
### Эти функции объединяют в себе все предыдущие, по сути, это упрощает запуск модели, назовём полный запуск циклом.



In [ ]:
#В функцию подаются все модели, 
#данные(путь к видео), 
#указывается тип данных(по стандарту - видео), 
#указываются параметри видео списком [секунда начала, секунда конца, пропуск кадров], по стандарту - [110, 130, 20]
#указывается необходимость запуска ансамбля(по стандарту вкл.), 
#и указывается "путь": "fast" или "accurate"(по стандарту "accurate")
result = detectors.run_full_cycle(models_all, "examples/3334137.mp4", ensemble=True, type="video", route="accurate")
result